## VAR vs VARMAX Model Validation


In this notebook, we will validate which model performs better given computational constraints: a VAR model retrained after each prediction or a VARMAX retrained every 20 predictions. To do so, we will retrun the model and functions constructed in [Systems_Identification_Fitting](Systems_Identification_Fitting.ipynb).

In [1]:
# import libraries
import pandas as pd
import numpy as np
from scipy import stats
import math as m
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, VARMAX
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")

%matplotlib inline 

os.chdir('..')

states = pd.read_csv('data/states.csv')
del states['Unnamed: 0']
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RaiDrawnFromSAFEs,collateral,debt,EthInUniswap,RaiInUniswap,ETH Price (OSM),block_timestamp
0,0.002481,4.362594,11860755,0.0,2788.522358,115792089237316195423570985008687.9,0.068533,3,1.0,1.0,1.0,3.14,2788.440433,14.89236,2788.385291,1.679713,676.933727,1803.656430,2021-02-15 10:16:16+00:00
1,0.002481,4.419266,11861008,0.0,2788.657935,115792089237316195423570985008687.9,0.108751,3,1.0,1.0,1.0,3.14,2788.440433,14.89236,2788.385291,1.679713,676.933727,1763.974936,2021-02-15 11:13:28+00:00
2,0.002481,4.419266,11861237,0.0,2788.657935,115792089237316195423570985008687.9,0.108751,3,1.0,1.0,1.0,3.14,2788.440433,14.89236,2788.385291,1.679713,676.933727,1763.974936,2021-02-15 12:07:36+00:00
3,0.001769,3.215446,11861502,0.0,3048.645955,115792089237316195423570985008687.9,0.116162,3,1.0,1.0,1.0,3.14,3048.440433,16.89236,3048.358490,1.418569,801.933727,1775.977923,2021-02-15 13:07:59+00:00
4,0.001769,3.215446,11861791,0.0,3048.645955,115792089237316195423570985008687.9,0.116162,3,1.0,1.0,1.0,3.14,3048.440433,16.89236,3048.358490,1.418569,801.933727,1805.792735,2021-02-15 14:07:59+00:00


In [2]:
# add additional state variables
states['RedemptionPriceinEth'] = states['RedemptionPrice'] / states['ETH Price (OSM)']
states['RedemptionPriceError'] = states['RedemptionPrice'] - states['marketPriceUsd']

In [3]:
# define constants (will come from cadCAD model but added here for calculations)
params = {
    'liquidation_ratio': 1.45,
    'debt_ceiling': 1e9,
    'uniswap_fee': 0.003,
    'arbitrageur_considers_liquidation_ratio': True,
}


<!-- ## Create Arbtrageur data vector $u^*$ -->

In [4]:
# subset state variables for arbitrageur vector
state_subset = states[['collateral','RaiDrawnFromSAFEs','RaiInUniswap','EthInUniswap']]

# map state data to vector fields 
state_subset.columns = ['Q','D','Rrai','Reth']

# alpha is the smoothing factor
local = state_subset.ewm(alpha=0.8).mean()
local

,Q,D,Rrai,Reth
0,14.892360,2.788440e+03,6.769337e+02,1.679713
1,14.892360,2.788440e+03,6.769337e+02,1.679713
2,14.892360,2.788440e+03,6.769337e+02,1.679713
3,16.494925,2.996774e+03,7.770940e+02,1.470463
4,16.812975,3.038120e+03,7.969721e+02,1.428934
...,...,...,...,...
2314,1083.900534,1.084647e+07,6.003190e+06,6575.363667
2315,1083.900470,1.084753e+07,6.038279e+06,6537.221037
2316,1083.900457,1.084646e+07,6.039119e+06,6559.444915
2317,1083.900455,1.085313e+07,6.093610e+06,6508.444876


In [5]:
# function to create coordinate transformations
def coordinate_transformations(params,df,Q,R_eth,R_rai,D,RedemptionPrice,EthPrice):
    '''
    Description:
    Function that takes in pandas dataframe and the names of columns
    
    Parameters:
    df: pandas dataframe containing states information
    Q: dataframe column name
    R_eth: dataframe column name
    R_rai: dataframe column name
    D: dataframe column name
    RedemptionPrice: dataframe column name
    EthPrice: dataframe column name

    Returns: Pandas dataframe with alpha, beta, gamma, delta transformed values
    
    Example:
    
    coordinate_transformations(params,states,'collateral','EthInUniswap','RaiInUniswap',
                           'RaiDrawnFromSAFEs','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]
    '''
    
    # Calculate alpha
    d = df[D].diff()
    d.fillna(0,inplace=True)
    df['d'] = d
    
    df['alpha'] = df['d'] / params['debt_ceiling']

    # calculate beta
    df['C_o'] = (df[RedemptionPrice]/states[EthPrice]) * params['liquidation_ratio']

    q = df[Q].diff()
    q.fillna(0,inplace=True)
    df['q'] = q


    df['C_1'] = (df['C_o'] * df[D]) - df[Q]

    df['beta'] = (df['q'] - (df['C_o']*df['d']))/ df['C_1']

    # calculate gamma
    r = df[R_rai].diff()
    r.fillna(0,inplace=True)
    df['r'] = r

    df['gamma'] = df['r']/df[R_rai]

    # calculate delta
    z = df[R_eth].diff()
    z.fillna(0,inplace=True)
    df['z'] = z

    df['delta'] = df['z']/df[R_eth]
    
    return df

In [6]:
transformed = coordinate_transformations(params,states,'collateral','EthInUniswap','RaiInUniswap',
                           'RaiDrawnFromSAFEs','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]

transformed

,alpha,beta,gamma,delta
0,0.000000e+00,-0.000000,0.000000,0.000000
1,0.000000e+00,-0.000000,0.000000,0.000000
2,0.000000e+00,-0.000000,0.000000,0.000000
3,2.600000e-07,-0.146901,0.155873,-0.184090
4,0.000000e+00,-0.000000,0.000000,0.000000
...,...,...,...,...
2314,3.654143e-06,-0.000359,0.009173,-0.009223
2315,0.000000e+00,0.000000,0.005464,-0.005587
2316,-1.604192e-06,0.000158,-0.001279,0.005684
2317,8.600000e-06,-0.000846,0.011119,-0.010670


In [7]:
# add additional signals to arbitrageur state
local['RedemptionPrice'] = states['RedemptionPrice']
local['ETH Price (OSM)'] = states['ETH Price (OSM)']

local

,Q,D,Rrai,Reth,RedemptionPrice,ETH Price (OSM)
0,14.892360,2.788440e+03,6.769337e+02,1.679713,3.140000,1803.656430
1,14.892360,2.788440e+03,6.769337e+02,1.679713,3.140000,1763.974936
2,14.892360,2.788440e+03,6.769337e+02,1.679713,3.140000,1763.974936
3,16.494925,2.996774e+03,7.770940e+02,1.470463,3.140000,1775.977923
4,16.812975,3.038120e+03,7.969721e+02,1.428934,3.140000,1805.792735
...,...,...,...,...,...,...
2314,1083.900534,1.084647e+07,6.003190e+06,6575.363667,3.007422,2702.280000
2315,1083.900470,1.084753e+07,6.038279e+06,6537.221037,3.007430,2757.389589
2316,1083.900457,1.084646e+07,6.039119e+06,6559.444915,3.007431,2773.590000
2317,1083.900455,1.085313e+07,6.093610e+06,6508.444876,3.007393,2761.044990


In [8]:
transformed_arbitrageur = coordinate_transformations(params,local,'Q','Reth','Rrai',
                           'D','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]

transformed_arbitrageur

,alpha,beta,gamma,delta
0,0.000000e+00,-0.000000,0.000000,0.000000
1,0.000000e+00,-0.000000,0.000000,0.000000
2,0.000000e+00,-0.000000,0.000000,0.000000
3,2.083333e-07,-0.121249,0.128891,-0.142303
4,4.134656e-08,-0.023359,0.024942,-0.029063
...,...,...,...,...
2314,5.311448e-06,-0.000522,0.007211,-0.006817
2315,1.062290e-06,-0.000105,0.005811,-0.005835
2316,-1.070896e-06,0.000105,0.000139,0.003388
2317,6.665821e-06,-0.000656,0.008942,-0.007836


In [9]:
def create_transformed_errors(transformed_states,transformed_arbitrageur):
    '''
    Description:
    Function for taking two pandas dataframes of transformed states and taking the difference
    to produce an error dataframe. 
    
    Parameters:
    transformed_states: pandas dataframe with alpha, beta, gamma, and delta features
    transformed_arbitrageur: pandas dataframe with alpha, beta, gamma, and delta features

    Returns:
    error pandas dataframe and transformation object
    
    '''
    alpha_diff = transformed_states['alpha'] - transformed_arbitrageur['alpha']
    beta_diff = transformed_states['beta'] - transformed_arbitrageur['beta']
    gamma_diff = transformed_states['gamma'] - transformed_arbitrageur['gamma']
    delta_diff = transformed_states['delta'] - transformed_arbitrageur['delta']


    e_u = pd.DataFrame(alpha_diff)
    e_u['beta'] = beta_diff
    e_u['gamma'] = gamma_diff
    e_u['delta'] = delta_diff

    e_u = e_u.astype(float)
    
    return e_u

e_u = create_transformed_errors(transformed, transformed_arbitrageur)
e_u.head()

,alpha,beta,gamma,delta
0,0.000000e+00,0.000000,0.000000,0.000000
1,0.000000e+00,0.000000,0.000000,0.000000
2,0.000000e+00,0.000000,0.000000,0.000000
3,5.166667e-08,-0.025652,0.026982,-0.041788
4,-4.134656e-08,0.023359,-0.024942,0.029063


In [10]:
def power_transformation(e_u):
    '''
    Definition:
    Function to perform a power transformation on the coordinate 
    transformed differenced data
    
    Parameters:
    e_u: Dataframe of coordinated transformed differenced data
    
    Required:
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    Returns:
    Transformed dataframe and transformation object
    
    Example:
    transformed_df, pt = power_transformation(e_u)
    '''
    pt = PowerTransformer()
    yeo= pd.DataFrame(pt.fit_transform(e_u),columns=e_u.columns)

    return yeo, pt

In [11]:
e_u_yeo, pt = power_transformation(e_u)

<!-- ### Autogressive lag selection -->

In [12]:
# split data between train and test (in production deployment, can remove)
split_point = int(len(e_u) * .8)
train = e_u_yeo.iloc[0:split_point]
test = e_u_yeo.iloc[split_point:]


states_train = states.iloc[0:split_point]
states_test = states.iloc[split_point:]

In [13]:
def VARMAX_prediction(e_u,RedemptionPriceError,newRedemptionPriceError,steps=1,lag=1):
    '''
    Description:
    Function to train and forecast a VARMAX model one step into the future
    
    Parameters:
    e_u: errors pandas dataframe
    RedemptionPriceErrorPrevious: 1d Numpy array of RedemptionPriceError values
    newRedemptionPriceError: exogenous latest redemption price error signal - float
    steps: Number of forecast steps. Default is 1
    lag: number of autoregressive lags. Default is 1
    
    Returns:
    Numpy array of transformed state changes
    
    Example
    Y_pred = VARMAX_prediction(train,states_train['RedemptionPriceError'],
                  states_test['RedemptionPriceError'][0:5],steps=5,lag=1)
    '''
    # instantiate the VARMAX model object from statsmodels 
    model = VARMAX(endog=e_u.values,exog=RedemptionPriceError,
                   initialization='approximate_diffuse',measurement_error=True)

    # fit model with determined lag values
    results = model.fit(order=(lag,0))
    
    Y_pred = results.forecast(steps = steps, exog=newRedemptionPriceError)
    
    return Y_pred.values

In [14]:
def VAR_prediction(e_u,lag=1):
    '''
    Description:
    Function to train and forecast a VAR model one step into the future
    
    Parameters:
    e_u: errors pandas dataframe
    lag: number of autoregressive lags. Default is 1
    
    Returns:
    Numpy array of transformed state changes
    
    Example
    VAR_prediction(e_u,6)    
    '''
    # instantiate the VAR model object from statsmodels 
    model = VAR(e_u)

    # fit model with determined lag values
    results = model.fit(lag)
    
    lag_order = results.k_ar
    
    Y_pred = results.forecast(e_u.values[-lag_order:],1)

    
    return Y_pred[0]

In [15]:
varmax_predictions = VARMAX_prediction(train,states_train['RedemptionPriceError'],
                  states_test['RedemptionPriceError'].values[0:20],steps=20,lag=1)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           38     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.73915D+00    |proj g|=  3.38343D-01

At iterate    5    f=  5.18992D+00    |proj g|=  5.78756D-02

At iterate   10    f=  5.15772D+00    |proj g|=  3.42084D-02

At iterate   15    f=  5.15390D+00    |proj g|=  6.49354D-03

At iterate   20    f=  5.15365D+00    |proj g|=  4.84560D-03

At iterate   25    f=  5.15360D+00    |proj g|=  8.50179D-04

At iterate   30    f=  5.15359D+00    |proj g|=  8.41480D-04

At iterate   35    f=  5.15358D+00    |proj g|=  2.18252D-04

At iterate   40    f=  5.15358D+00    |proj g|=  5.92292D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg 

In [16]:
var_predictions = []
train_var = train.copy()
for i in range(0,20): 
    var_predictions.append(VAR_prediction(train_var,15))
    train_var = train_var.append(test.iloc[i])

In [17]:
def invert_power_transformation(pt,prediction):
    '''
    Definition:
    Function to invert power transformation
    
    Parameters:
    pt: transformation object
    prediction: Numpy array of model state coordinate transformed percentage changes
    
    Required:
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    Returns:
    inverted transformation numpy array
    
    Example:
    inverted_array = invert_power_transformation(pt,prediction)
    
    '''
    # transform back into coordinate system
    inverted = pt.inverse_transform(prediction)
    
    return inverted

In [18]:
# invert the power transformation
var_predictions = invert_power_transformation(pt,var_predictions)
varmax_predictions = invert_power_transformation(pt,varmax_predictions)

## Model Evaluation Comparision (in coordinate transformed and power transformed differenced space)  

In [19]:
var_predictions_df = pd.DataFrame(var_predictions,columns=['alpha','beta','gamma','delta'])
varmax_predictions_df = pd.DataFrame(varmax_predictions,columns=['alpha','beta','gamma','delta'])

## Alpha - Root Mean Square Error

In [20]:
var_alpha_rmse = m.sqrt(mean_squared_error(var_predictions_df['alpha'], test.head(20)['alpha']))
varmax_alpha_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['alpha'], test.head(20)['alpha']))
if var_alpha_rmse >= varmax_alpha_rmse:
    print('VARMAX performs better by {}'.format(varmax_alpha_rmse-var_alpha_rmse))
else:
    print('VAR performs better by {}'.format(var_alpha_rmse-varmax_alpha_rmse))

VAR performs better by -4.099296346510295e-06


## Beta - Root Mean Square Error

In [21]:
var_beta_rmse = m.sqrt(mean_squared_error(var_predictions_df['beta'], test.head(20)['beta']))
varmax_beta_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['beta'], test.head(20)['beta']))
if var_beta_rmse >= varmax_beta_rmse:
    print('VARMAX performs better by {}'.format(varmax_beta_rmse-var_beta_rmse))
else:
    print('VAR performs better by {}'.format(var_beta_rmse-varmax_beta_rmse))

VAR performs better by -0.0003539049044675169


## Gamma - Root Mean Square Error

In [22]:
var_gamma_rmse = m.sqrt(mean_squared_error(var_predictions_df['gamma'], test.head(20)['gamma']))
varmax_gamma_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['gamma'], test.head(20)['gamma']))
if var_gamma_rmse >= varmax_gamma_rmse:
    print('VARMAX performs better by {}'.format(varmax_gamma_rmse-var_gamma_rmse))
else:
    print('VAR performs better by {}'.format(var_gamma_rmse-varmax_gamma_rmse))

VAR performs better by -0.00036563194191302495


## Delta - Root Mean Square Error

In [23]:
var_delta_rmse = m.sqrt(mean_squared_error(var_predictions_df['delta'], test.head(20)['delta']))
varmax_delta_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['delta'], test.head(20)['delta']))
if var_delta_rmse >= varmax_delta_rmse:
    print('VARMAX performs better by {}'.format(varmax_delta_rmse-var_delta_rmse))
else:
    print('VAR performs better by {}'.format(var_delta_rmse-varmax_delta_rmse))

VARMAX performs better by -0.00025588148090463503


## Model Evaluation Comparision as States

In [24]:
def inverse_transformation_and_state_update(Y_pred,previous_state,params):
    '''
    Description:
    Function to take system identification model prediction and invert transfrom and create new state
    
    Parameters:
    y_pred: numpy array of transformed state changes
    previous_state: pandas dataframe of previous state or 'current' state
    params: dictionary of system parameters
    
    Returns:
    pandas dataframe of new states 
    
    Example:
    inverse_transformation_and_state_update(Y_pred,previous_state,params)
    '''
    
    d_star = Y_pred[0] * params['debt_ceiling']
    
    q_star = previous_state['C_o'] * params['debt_ceiling'] * Y_pred[0] + previous_state['C_1'] * Y_pred[1]
    
    r_star = Y_pred[2] * previous_state['gamma'] * previous_state['RaiInUniswap']

    z_star = Y_pred[3] * previous_state['delta'] * previous_state['EthInUniswap']

    new_state = pd.DataFrame(previous_state[['collateral','EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']].to_dict())
    new_state['Q'] = new_state['collateral'] + q_star
    new_state['D'] = new_state['RaiDrawnFromSAFEs'] + d_star
    new_state['R_Rai'] = new_state['RaiInUniswap'] + r_star
    new_state['R_Eth'] = new_state['EthInUniswap'] + z_star
    
    return new_state[['Q','D','R_Rai','R_Eth']]



In [25]:
VAR_new_states = []
index = -21
for i in range(0,20): 
    previous_state = states.iloc[train_var.index[index:index+1]]
    VAR_new_states.append(inverse_transformation_and_state_update(var_predictions[i],previous_state,params))
    index += 1
    
VAR_new_states = pd.concat(VAR_new_states)
VAR_new_states

,Q,D,R_Rai,R_Eth
1854,1507.637702,2.941844e+07,1.966459e+07,19142.963416
1855,1446.553382,2.950175e+07,1.995888e+07,18860.541566
1856,1445.090491,2.967998e+07,2.004081e+07,18782.018962
1857,1488.790971,2.982481e+07,2.014574e+07,18670.488103
1858,1487.163556,2.988804e+07,2.013334e+07,18734.135232
1859,1439.680002,2.989775e+07,2.012070e+07,18738.958560
1860,1466.194280,2.990544e+07,2.014744e+07,18711.730634
1861,1375.782913,2.996199e+07,2.028709e+07,18574.872820
1862,1447.066094,3.007227e+07,2.036559e+07,18502.083653
1863,1462.401618,3.008141e+07,2.015797e+07,18692.246570


In [26]:
VARMAX_new_states = []
index = -21
for i in range(0,20): 
    previous_state = states.iloc[train_var.index[index:index+1]]
    VARMAX_new_states.append(inverse_transformation_and_state_update(varmax_predictions[i],previous_state,params))
    index += 1
    
VARMAX_new_states = pd.concat(VARMAX_new_states)
VARMAX_new_states

,Q,D,R_Rai,R_Eth
1854,1464.904466,2.941526e+07,1.966459e+07,19142.977075
1855,1467.215794,2.950312e+07,1.995934e+07,18861.215858
1856,1466.832579,2.970178e+07,2.004079e+07,18782.034844
1857,1456.923083,2.984548e+07,2.014579e+07,18670.361654
1858,1455.329949,2.988853e+07,2.013334e+07,18734.168550
1859,1455.859837,2.988948e+07,2.012070e+07,18738.962540
1860,1455.737035,2.988949e+07,2.014743e+07,18711.741142
1861,1457.003720,2.995944e+07,2.028707e+07,18575.117023
1862,1456.860062,3.008084e+07,2.036553e+07,18502.092212
1863,1458.234242,3.008088e+07,2.015833e+07,18692.720280


In [27]:
test_data = states.iloc[test.index[0:20]][['collateral','EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']]


In [28]:
test_data

,collateral,EthInUniswap,RaiInUniswap,RaiDrawnFromSAFEs
1855,1463.4226,18861.221200,1.995936e+07,2.950289e+07
1856,1463.4226,18782.037267,2.004080e+07,2.970079e+07
1857,1453.4226,18670.364620,2.014579e+07,2.984478e+07
1858,1453.4226,18734.165150,2.013334e+07,2.988766e+07
1859,1453.4226,18738.962359,2.012070e+07,2.988866e+07
1860,1453.4226,18711.742367,2.014743e+07,2.988866e+07
1861,1453.4226,18575.119941,2.028708e+07,2.995866e+07
1862,1453.4226,18502.094383,2.036554e+07,3.008006e+07
1863,1453.4226,18692.719606,2.015832e+07,3.008015e+07
1864,1453.4226,18655.017968,2.018636e+07,3.008515e+07


## Q - Root Mean Square Error

In [29]:
var_q_rmse = m.sqrt(mean_squared_error(VAR_new_states['Q'], test_data['collateral']))
varmax_q_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['Q'], test_data['collateral']))
if var_q_rmse >= varmax_q_rmse:
    print('VARMAX performs better by {}'.format(varmax_q_rmse-var_q_rmse))
else:
    print('VAR performs better by {}'.format(var_q_rmse-varmax_q_rmse))

VARMAX performs better by -33.978862360146636


## D - Root Mean Square Error

In [30]:
var_d_rmse = m.sqrt(mean_squared_error(VAR_new_states['D'], test_data['RaiDrawnFromSAFEs']))
varmax_d_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['D'], test_data['RaiDrawnFromSAFEs']))
if var_d_rmse >= varmax_d_rmse:
    print('VARMAX performs better by {}'.format(varmax_d_rmse-var_d_rmse))
else:
    print('VAR performs better by {}'.format(var_d_rmse-varmax_d_rmse))

VARMAX performs better by -2598.711131820921


## R_Rai - Root Mean Square Error

In [31]:
var_rai_rmse = m.sqrt(mean_squared_error(VAR_new_states['R_Rai'], test_data['RaiInUniswap']))
varmax_rai_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['R_Rai'], test_data['RaiInUniswap']))
if var_rai_rmse >= varmax_rai_rmse:
    print('VARMAX performs better by {}'.format(varmax_rai_rmse-var_rai_rmse))
else:
    print('VAR performs better by {}'.format(var_rai_rmse-varmax_rai_rmse))

VAR performs better by -2.627722575212829


## R_Eth - Root Mean Square Error

In [32]:
var_eth_rmse = m.sqrt(mean_squared_error(VAR_new_states['R_Eth'], test_data['EthInUniswap']))
varmax_eth_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['R_Eth'], test_data['EthInUniswap']))
if var_eth_rmse >= varmax_eth_rmse:
    print('VARMAX performs better by {}'.format(varmax_eth_rmse-var_eth_rmse))
else:
    print('VAR performs better by {}'.format(var_eth_rmse-varmax_eth_rmse))

VAR performs better by -0.17702298110404513


In [33]:
print('Aggregated VARMAX RMSE in absolute values:')
print(abs(varmax_eth_rmse) + abs(varmax_rai_rmse) + abs(varmax_d_rmse) + abs(varmax_q_rmse))

print('Aggregated VAR RMSE in absolute values:')
print(abs(var_eth_rmse) + abs(var_rai_rmse) + abs(var_d_rmse) + abs(var_q_rmse))

Aggregated VARMAX RMSE in absolute values:
221293.33294602376
Aggregated VAR RMSE in absolute values:
223923.2181946485


### Exponential weighted moving average test grid

#### Aggregated RMSE values with VARMAX(1) and VAR(6)
VAR lags changed between alphas but for simplicity we held constant

|Alpha   |VARMAX RMSE   | VAR RMSE  | 
|---|---|---|
|0.1   |346,906.48   |356,511.48   |
|0.2  |355,980.48   |353,095.32   |
|0.3  |352,226.20   |350,923.13   |
|0.4  |347,489.97   |348,758.57  |
|0.5  |348,488.02  |347,155.68   |
|0.6  |348,345.53   |346,199.91   |
|0.7  |347,916.76   |345,765.18   |
|**0.8**  |348,061.60   |**345,692.82**   |
|0.9  |349,104.38   |346,223   |

## Conclusion


In this notebook we evaluated the performance of a VARMAX(1) model trained every 20 timesteps an the exogenous signal of redemption price error, against a VAR(15) model retrained everytime step. We decided on using a VAR(15) for the Rai Digital Twin because it performs slightly better, being able to retrain at every prediction timestep. 